In [11]:
import json
import pandas as pd

In [12]:
with open('C:/CODE/SIC_BigData_Real-Estate-Price-Prediction/data/Dat/Dat_raw-dataset.json','r',encoding='utf-8') as file:
    data = json.load(file)
df1 = pd.json_normalize(data)
df1.drop_duplicates(inplace=True)

In [13]:
df1.drop(columns=['DiaChi','City','District','PhapLy','HuongNha','NoiThat'])

,DienTich,MucGia,MatTien,DuongVao,Lat,Long,HuongBanCong,SoPhongNgu,SoTang,SoToilet
0,180 m²,Thỏa thuận,5 m,7 m,11.001454,106.818515,NaN,NaN,NaN,NaN
1,29.235 m²,125 tỷ,55 m,4 m,11.538185,107.800136,NaN,NaN,NaN,NaN
2,250 m²,180 triệu/m²,11 m,NaN,10.762480,106.693775,NaN,NaN,NaN,NaN
3,63 m²,125 triệu/m²,"4,5 m",NaN,20.842289,106.704846,NaN,NaN,NaN,NaN
4,3.000 m²,"4,5 tỷ",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
40038,340 m²,"5,5 tỷ",20 m,7 m,NaN,NaN,NaN,NaN,NaN,NaN
40039,100 m²,"1,6 tỷ","5,5 m",10 m,22.663841,106.234955,NaN,NaN,NaN,NaN
40040,72 m²,"1,6 tỷ","4,5 m",6 m,NaN,NaN,NaN,NaN,NaN,NaN
40041,300 m²,"5,8 tỷ",15 m,7 m,22.671997,106.218518,NaN,NaN,NaN,NaN


In [14]:
#xử lý diện tích
df1["DienTich"] = df1["DienTich"].str.replace(' m²','').str.replace('.', '').str.replace(',', '.').str.strip()

In [15]:
#Hàm tính toán giá Đất
def clean_price(row):
    price = row['MucGia']
    area = row['DienTich']

    # Đổi kiểu cột DienTich thành float
    area = float(area)

    # Đảm bảo price là chuỗi
    if isinstance(price, (float, int)):
        return price

    price = str(price)

    #chuyển đổi đơn vị giá -> đồng
    if 'triệu/m²' in price:
        price_value = float(price.replace(' triệu/m²', '').replace(',', '.').strip()) * 1000000
        return price_value * area
    elif 'thỏa thuận' in price:
        return price
    elif 'nghìn/m²' in price:
        price_value = float(price.replace(' nghìn/m²', '').replace(',', '.').strip()) * 1000
        return price_value * area
    elif 'tỷ/m²' in price:
        price_value = float(price.replace(' tỷ/m²', '').replace(',', '.').strip()) * 1000000000
        return price_value * area
    elif 'triệu' in price:
        price_value = float(price.replace(' triệu', '').replace(',', '.').strip()) * 1000000
        return price_value
    elif 'tỷ' in price:
        price_value = float(price.replace(' tỷ', '').replace(',', '.').strip()) * 1000000000
        return price_value
    elif 'nghìn' in price:
        price_value = float(price.replace(' nghìn', '').replace(',', '.').strip()) * 1000
        return price_value

    return price

# Áp dụng hàm vào DataFrame
df1['MucGia'] = df1.apply(clean_price, axis=1)
print(df1['MucGia'])

0            Thỏa thuận
1        125000000000.0
2         45000000000.0
3          7875000000.0
4          4500000000.0
              ...      
40038      5500000000.0
40039      1600000000.0
40040      1600000000.0
40041      5800000000.0
40042      6000000000.0
Name: MucGia, Length: 23891, dtype: object


In [16]:
#Gán giá trị trung vị cho 'thỏa thuận'
# Tính giá trị trung vị
median_price = df1[df1['MucGia'] != 'Thỏa thuận']['MucGia'].astype(float).median()

# Thay thế giá trị "thỏa thuận"
df1['MucGia'] = df1['MucGia'].replace('Thỏa thuận', median_price)

C:\Users\quanp\AppData\Local\Temp\ipykernel_10340\1240423143.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['MucGia'] = df1['MucGia'].replace('Thỏa thuận', median_price)


In [17]:
#xử lý diện tích mặt tiền
df1["MatTien"] = df1["MatTien"].str.replace(' m','').str.replace(',','.').str.strip()

#xử lý diện tích đường vào
df1["DuongVao"] = df1["DuongVao"].str.replace(' m','').str.replace(',','.').str.strip()

#xử lý diện tích số phòng ngủ
df1["SoPhongNgu"] = df1["SoPhongNgu"].str.replace(' phòng','').str.strip()

#xử lý số phòng Toilet
df1["SoToilet"] = df1["SoToilet"].str.replace(' phòng','').str.strip()

#xử lý số tầng
df1["SoTang"] = df1["SoTang"].str.replace(' tầng','').str.strip()